In [1]:
import torch
import torchvision
import numpy as np

In [2]:
torch.__version__,np.__version__ 

('1.12.1', '1.23.1')

## Dataset Study

### Dataset이란?
어떠한 AI model 을 학습 시키기 위해서 필요한 Data들을 모아놓은 데이터 셋

종류
* pytorch에서 제공해주는 기본 Dataset
* dataset (img, label) 이 '파일'을 받아서 직접 만드는 Custom Dataset이 존재

### 1. pytorch에서 기본적으로 제공해주는 Dataset을 만져보기

### CIFAR10 Dataset
Classfication - Image 한 장을 보고 이 Image 가 어떤 사진인지 확인하는 task
ex) 강아지가 있는 사진을 보고 AI model 은 강아지! 라고 prediction 하는 것이다.

이 Classfication을 위해서 만들어진 Dataset - CIFAR10, CIFAR100

In [ ]:
cifar10 = torchvision.datasets.CIFAR10(root='.',train=True, download=True)

Dataset을 한번 다운로드 했다면, download를 False로 놓고 진행하면 됨

In [3]:
train_cifar_2 = torchvision.datasets.CIFAR10(root='.', train=True, download=False)

## Img, label은 tensor 형태를 가져야한다. 
dataset을 선언할 때, transform을 넣어주지 않으먄, Imge 자체가 return 되기 때문에 문제가 된다.

Image 자체를 원하는게 아니고, 이 Image를 Tensor 형태로 바꾼 값을 원하는 (type을 바꾼다) 것.
그렇기 떄문에 Dataset을 선언할 때 우리에게 뿌려줄 때 이 transform을 먹여서 tensor 형태로 변경 후 뿌려달라는 것을 넣어줘야한다.

In [ ]:
## 부탁 할 transform 선언
## Compose는 n개의 transform을 한번에 묶어주는 역할
transform = torchvision.transforms.Cobmpose([torchvision.transforms.ToTensor()])

In [ ]:
## transform을 먹히게 만든 dataset을 선언
train_cifar10 = torchvision.datasets.CIFAR10(root='.', train = True, transform=transform)

### train set, validation set
train_set : AI model train을 위한 데이터 셋
validation_set | test_set : AI model이 처음 보는 이미지를 보고 성적을 매길 수 있게 만든 데이터 셋

예를 들어서 1000개의 사진을 갖고 있다. - 900개는 training 용으로 사용, 100 validation 용으로 사용
validation set을 학습 시킨다는 의미 - 시험을 보는데 시험지를 가지고 공부를 한다? - 평가 점수는 올라가겠지만, 공정하지 않기 때문에 아무도 인정하지 않는다.

dataset은 dataset이라는 Class로 감싸져 있고 학습용으로 만들기 위해서는
우리가 1000개의 이미지를 가지고 있다. -> 모델에 몇 장씩 training 시킬 것인가?

이 몇장씩을 나눠주는 친구를 Dataloader라고 한다. Dataloader 가 Dataset을 감싸고 있는 형태라고 생각하면 편하다.

### Train dataset 확인
Dataset을 생성한 뒤에는, 우리가 원하는 대로 data가 뽑혀 나오는지 확인해야하는 과정이 필수로 들어간다.

In [ ]:
import matplotlib.pyplot as plt #사진을 보여주기 위한 library

기본적인 Dataset (torchvision에서 제공해주는) 어떻게 data가 뽑아져 나오냐면, (img, label) 쌍으로 return 된다.

In [ ]:
img, label = train_cifar10.__getitem__(7)

In [ ]:
label_name = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
print(type(img), type(label), img.shape, label_name[label])

In [ ]:
## plt를 사용할 때는 -> torch를 지원을 안하고 numpy만 지원한다.
img_for_show = img.permute(1,2,0).numpy()
#print(type(img_for_show))
#print(img_for_show)

In [ ]:
plt.imshow(img_for_show)
plt.title(label_name[label])

## AI model 을 만들어보자!
AI model은 굉장히 많은 종류가 있다. Convolution network, Fully Connected Network, ... 등

우리는 이 ML-Study에서 Convolution base model만을 사용할 것

model은 torch에서 제공하는 torch.nn 라이브러리를 이용하여 작성합니다.

In [ ]:
import torch.nn as nn

### convolution
convolution 연산은 filter가 이전 feature를 훑으면서 새로운 featrue을 해석한다 라고 생각하면 된다.

Convblock : convolution -> BN (Batch Nomalization) -> RELU (Activaition)

In [ ]:
convblock1 = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, stride=2, padding=0), 
    nn.BatchNorm2d(num_features=32),
    nn.ReLU()
)

dummy_input = torch.zeros(2, 3, 32, 32) #model input shape : [B, C, H, W] [Batch, Channel, Height, Width] Batch : n장의 이미지
out_convblock1 = convblock1(dummy_input)

#print("out convblock shape : ", out_convblock1.shape)
print(f"out convblock shape : {out_convblock1.shape}")


In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.convblock1 = nn.Sequential(
                                            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, stride=2, padding=0), 
                                            nn.BatchNorm2d(num_features=32),
                                            nn.ReLU())
        self.convblock2 = nn.Sequential(
                                            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1), 
                                            nn.BatchNorm2d(num_features=64),
                                            nn.ReLU())
        self.convblock3 = nn.Sequential(
                                            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1), 
                                            nn.BatchNorm2d(num_features=128),
                                            nn.ReLU())
        self.fc1 = nn.Sequential(
                                        nn.Linear(in_features=2048, out_features=64),
                                        nn.BatchNorm1d(64),
                                        nn.ReLU())
        self.fc2 = nn.Sequential(
                                        nn.Linear(in_features=64, out_features=10))
    def forward(self, x):
        B,C,H,W = x.shape
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = x.view(B, -1)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        

In [ ]:
mymodel = MyModel()

In [ ]:
output = mymodel(dummy_input)

In [ ]:
output.shape

In [ ]:
pred = output.softmax(1)

In [ ]:
pred

In [ ]:
torch.argmax(pred, dim = 1)

In [ ]:
print(img.shape)
# dataset에서 getitem으로 바로 뽑았기 때문에, dataloader를 거치지 않고 B가 없음!
# Model의 인풋은 무조건 shape이 [B,C,H,W] 여야하기 때문에 B를 만들어준다!
img_input = img.unsqueeze(0)
print(img_input.shape)

In [ ]:
mymodel.eval()
model_out = mymodel(img_input)
pred = model_out.softmax(1)
print(pred)

pred_class = torch.argmax(pred, dim = 1)
print("pred class : ", label_name[pred_class])
print("real ground truth : ", label_name[label])